In [1]:
from model import *

In [2]:
if __name__ == '__main__':
    h_dim = 3
    max_length = 4
    num_heads = 12
    d_out = 7

    vocab_path = r'C:\Users\jaspe\PycharmProjects\PythonProject7\Debugging\Encoding\byte_training\tok.json'
    tokenizer = BytePairTokenizer(vocab_file=vocab_path)

    vocab_size = max(tokenizer.vocab.values()) + 2
    embedding_layer = nn.Embedding(vocab_size, h_dim, padding_idx=0)
    pos_encoding_layer = PositionalEncoding(max_length, h_dim)
    encoding_layer = Encoding(embedding_layer, pos_encoding_layer)
    stacked_attention = MyStackedFunction(
        MaskedSelfAttention, num_heads, h_dim, d_out, max_length, dropout=0.1)

    print("Running batch size validation test...")
    phrases = ['to pimp a butterfly', 'keep it moving forward', 'it is okay to be gay', 'money is not everything']
    input_ids = create_token_tensor(phrases, tokenizer, max_length)

    encoded_sequences = encoding_layer(input_ids)
    context, weights = stacked_attention(encoded_sequences)

    assert context.shape == (len(phrases), num_heads, max_length, d_out)
    assert weights.shape == (len(phrases), num_heads, max_length, max_length)
    print("✅ Batch size validation test passed.")
    print("-" * 20)

    print("\nRunning attention visualization test...")
    phrase = ['Abraham Lincoln']
    input_tensor = create_token_tensor(phrase, tokenizer, max_length)
    encoded_single_sequence = encoding_layer(input_tensor)
    context_1, weights_1 = stacked_attention(encoded_single_sequence)

    token_ids = input_tensor[0].tolist()
    string_tokens = [tokenizer.decode([i]) if i != 0 else "[PAD]" for i in token_ids]

    head_to_show = 2
    single_head_weights = weights_1[0, head_to_show, :, :]

    visualize_attention(
        tokens=string_tokens,
        attn_weights=single_head_weights,
        head_idx=head_to_show
    )
    print("✅ Attention visualization test finished.")

C:\Users\jaspe\PycharmProjects\PythonProject7\Debugging\Encoding\byte_training\tok.json
loaded a model with  4201 vocabulary!
target vocab size is:  None !
None
Running batch size validation test...
✅ Batch size validation test passed.
--------------------

Running attention visualization test...

--- Attention Head #2 ---
           Abraham    Lincoln   [PAD]     [PAD]   
──────────────────────────────────────────────────
Abraham      0.01      -inf      -inf      -inf   
 Lincoln    -0.01     -0.01      -inf      -inf   
[PAD]        0.06     -0.07     -0.05      -inf   
[PAD]       -0.02      0.06      0.08      0.27   

✅ Attention visualization test finished.


In [3]:
phrase = 'Abraham Lincoln'
ids = tokenizer.encode(phrase)

emb = encoding_layer(torch.tensor(ids).reshape(-1, 1))
attn = stacked_attention(emb)

In [4]:
attn[1].shape

torch.Size([2, 12, 1, 1])

In [5]:
ids

[53339, 50439]